<a href="https://colab.research.google.com/github/Liekeverhoeven262/Master-Thesis-Data-Science-Society/blob/main/Final_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Construct the path to the file in Google Drive
personal_data_path = '/content/drive/My Drive/Master Thesis/FINAL_personal_data_compact3without.xlsx'
thermal_data_path = '/content/drive/My Drive/Master Thesis/FINAL_scaled_features_df_max.xlsx'

# Load the dataset
personal_data_df = pd.read_excel(personal_data_path)
thermal_data_df = pd.read_excel(thermal_data_path)

In [ ]:
data = pd.merge(thermal_data_df, personal_data_df, on='ID')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
from imblearn.over_sampling import SMOTE

In [ ]:
# Define features and target
features = data.drop(columns=['ID', 'Cluster', 'Sum2'])
target = data['Sum2']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Apply SMOTE to handle imbalanced dataset
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
scaler = StandardScaler()
X_train_smote = scaler.fit_transform(X_train_smote)
X_test = scaler.transform(X_test)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Data voorbereiding
X = np.array(X_train_smote)
y = np.array(y_train_smote)

In [ ]:
!pip install scikeras

from scikeras.wrappers import KerasClassifier

In [ ]:
!pip install --upgrade scikit-learn imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1


In [ ]:
!pip uninstall scikit-learn -y
!pip uninstall imbalanced-learn -y

Found existing installation: scikit-learn 1.4.2
Uninstalling scikit-learn-1.4.2:
  Successfully uninstalled scikit-learn-1.4.2
Found existing installation: imbalanced-learn 0.7.0
Uninstalling imbalanced-learn-0.7.0:
  Successfully uninstalled imbalanced-learn-0.7.0


In [ ]:
!pip install -U scikit-learn
!pip install -U imbalanced-learn
!pip install -U scikeras

  Using cached imbalanced_learn-0.12.2-py3-none-any.whl (257 kB)
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.8.0
    Uninstalling imbalanced-learn-0.8.0:
      Successfully uninstalled imbalanced-learn-0.8.0


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, classification_report, precision_recall_curve, auc
from imblearn.over_sampling import SMOTE

In [ ]:
def create_model(learning_rate=0.01, optimizer='adam'):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),  # Standaard aantal neuronen
        Dropout(0.2),  # Standaard dropout rate
        Dense(32, activation='relu'),  # Standaard halvering van neuronen in de tweede laag
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = SGD(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[tf.keras.metrics.Recall()])
    return model

In [ ]:
# Data voorbereiding en SMOTE
def prepare_data(X_train, y_train):
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    scaler = StandardScaler()
    X_train_smote = scaler.fit_transform(X_train_smote)
    return X_train_smote, y_train_smote, scaler

model = KerasClassifier(model=create_model, verbose=0)

In [ ]:
# Parameter grid voor GridSearchCV
param_grid = {
    'model__learning_rate': [0.01, 0.1, 0.2, 0.3],
    'model__optimizer': ['adam', 'sgd'],
    'batch_size': [16, 32, 64],
    'epochs': [10, 50, 100]
}

In [ ]:
from sklearn.metrics import make_scorer, recall_score

#focussen op de positieve klasse
recall_scorer = make_scorer(recall_score, pos_label=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=recall_scorer, verbose=1)

# Train en test splits
features = data.drop(columns=['ID', 'Cluster', 'Sum2'])
target = data['Sum2']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Voorbereid data met SMOTE en normalisatie
X_train_smote, y_train_smote, scaler = prepare_data(X_train, y_train)

In [ ]:
# Uitvoeren van GridSearch
grid_result = grid.fit(X_train_smote, y_train_smote)
print("Best Recall: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Recall: 0.928164 using {'batch_size': 16, 'epochs': 50, 'model__learning_rate': 0.01, 'model__optimizer': 'adam'}


In [ ]:
# Evaluatie van het beste model op de testset
X_test = scaler.transform(X_test)
y_pred = grid_result.predict(X_test)
y_pred_prob = grid_result.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Low VVR', 'High VVR']))

precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
auc_pr = auc(recall, precision)
print("AUC-PR Score:", auc_pr)

Classification Report:
              precision    recall  f1-score   support

     Low VVR       0.85      0.95      0.90        43
    High VVR       0.87      0.65      0.74        20

    accuracy                           0.86        63
   macro avg       0.86      0.80      0.82        63
weighted avg       0.86      0.86      0.85        63

AUC-PR Score: 0.7602480096471037


## Including Cluster

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, classification_report, precision_recall_curve, auc
from imblearn.over_sampling import SMOTE

features = data.drop(columns=['ID', 'Sum2'])
target = data['Sum2']
data_with_cluster = pd.get_dummies(features, columns=['Cluster'], prefix='Cluster')

X_train_with_cluster, X_test_with_cluster, y_train_with_cluster, y_test_with_cluster = train_test_split(data_with_cluster, target, test_size=0.2, random_state=42)

def prepare_data(X_train, y_train):
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    scaler = StandardScaler()
    X_train_smote = scaler.fit_transform(X_train_smote)
    return X_train_smote, y_train_smote, scaler

def create_model_with_cluster(input_shape, learning_rate=0.01, optimizer='adam'):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = SGD(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[tf.keras.metrics.Recall()])
    return model

model_with_cluster = KerasClassifier(model=create_model_with_cluster, verbose=0, input_shape=X_train_with_cluster.shape[1])

param_grid_with_cluster = {
    'model__learning_rate': [0.01, 0.1, 0.2, 0.3],
    'model__optimizer': ['adam', 'sgd'],
    'batch_size': [16, 32, 64],
    'epochs': [10, 50, 100]
}

grid_with_cluster = GridSearchCV(estimator=model_with_cluster, param_grid=param_grid_with_cluster, cv=5, scoring='recall', verbose=1)

X_train_smote_with_cluster, y_train_smote_with_cluster, scaler_with_cluster = prepare_data(X_train_with_cluster, y_train_with_cluster)

grid_result_with_cluster = grid_with_cluster.fit(X_train_smote_with_cluster, y_train_smote_with_cluster)
print("Best Recall: %f using %s" % (grid_result_with_cluster.best_score_, grid_result_with_cluster.best_params_))

X_test_with_cluster = scaler_with_cluster.transform(X_test_with_cluster)
y_pred_with_cluster = grid_result_with_cluster.predict(X_test_with_cluster)
y_pred_prob_with_cluster = grid_result_with_cluster.predict_proba(X_test_with_cluster)[:, 1]

print("Classification Report with Cluster:")
print(classification_report(y_test_with_cluster, y_pred_with_cluster, target_names=['Low VVR', 'High VVR']))

precision_with_cluster, recall_with_cluster, _ = precision_recall_curve(y_test_with_cluster, y_pred_prob_with_cluster)
auc_pr_with_cluster = auc(recall_with_cluster, precision_with_cluster)
print("AUC-PR Score with Cluster:", auc_pr_with_cluster)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


Best Recall: 0.886275 using {'batch_size': 32, 'epochs': 50, 'model__learning_rate': 0.01, 'model__optimizer': 'adam'}
Classification Report with Cluster:
              precision    recall  f1-score   support

     Low VVR       0.78      0.84      0.81        43
    High VVR       0.59      0.50      0.54        20

    accuracy                           0.73        63
   macro avg       0.69      0.67      0.67        63
weighted avg       0.72      0.73      0.72        63

AUC-PR Score with Cluster: 0.7064785939079614
